# Library Load

In [1]:
# 필요한 PyTorch 라이브러리 불러오기
import os
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image

import PIL
from PIL import Image
import matplotlib.pyplot as plt

import cv2
import numpy as np

import xml.etree.ElementTree as ET

import copy

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
# GPU 장치 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Data Load

In [4]:
# Content Image Path
content_img_folder_path = './oriental_img/'
content_img_folder_list = os.listdir(content_img_folder_path)
content_img_folder_list

['carp', 'cranebird', 'deer', 'lotusflower', 'turtle']

In [5]:
# Style Image Path
style_img_path = './khd_style_img/'
style_img_list = os.listdir(style_img_path)
style_img_list

['b34.jpg', 'b41.jpg', 'b98.jpg']

In [6]:
style_img_name = style_img_list[0]
style_img_name

'b34.jpg'

In [7]:
# 뉴럴 네트워크 모델을 불러옵니다.
cnn = models.vgg19(pretrained=True).features.to(device).eval()
print(cnn)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo

# Function

In [8]:
# 이미지를 불러와 다운받아 텐서(Tensor) 객체로 변환하는 함수
def image_loader(img_path, imsize):
    loader = transforms.Compose([
        transforms.Resize(imsize), # 이미지의 크기를 변경
        transforms.ToTensor() # torch.Tensor 형식으로 변경 [0, 255] → [0, 1]
    ])
    image = PIL.Image.open(img_path)
    # 네트워크 입력에 들어갈 이미지에 배치 목적의 차원(dimension) 추가
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float) # GPU로 올리기


# torch.Tensor 형태의 이미지를 화면에 출력하는 함수
def imshow(tensor):
    # matplotlib는 CPU 기반이므로 CPU로 옮기기
    image = tensor.cpu().clone()
    # torch.Tensor에서 사용되는 배치 목적의 차원(dimension) 제거
    image = image.squeeze(0)
    # PIL 객체로 변경 
    image = transforms.ToPILImage()(image)
    # 이미지를 화면에 출력(matplotlib는 [0, 1] 사이의 값이라고 해도 정상적으로 처리)
    #plt.imshow(image)
    #plt.show()

In [9]:
# 입력 정규화(Normalization)를 위한 초기화
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

class Normalization(nn.Module):
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        self.mean = mean.clone().view(-1, 1, 1)
        self.std = std.clone().view(-1, 1, 1)

    def forward(self, img):
        return (img - self.mean) / self.std

In [10]:
def gram_matrix(input):
    # a는 배치 크기, b는 특징 맵의 개수, (c, d)는 특징 맵의 차원을 의미
    a, b, c, d = input.size()
    # 논문에서는 i = 특징 맵의 개수, j = 각 위치(position)
    features = input.view(a * b, c * d)
    # 행렬 곱으로 한 번에 Gram 내적 계산 가능
    G = torch.mm(features, features.t())
    # Normalize 목적으로 값 나누기
    return G.div(a * b * c * d)


# 스타일 손실(style loss) 계산을 위한 클래스 정의
class StyleLoss(nn.Module):
    def __init__(self, target_feature):
        super(StyleLoss, self).__init__()
        self.target = gram_matrix(target_feature).detach()

    def forward(self, input):
        G = gram_matrix(input)
        self.loss = F.mse_loss(G, self.target)
        return input
    

# 콘텐츠 손실(content loss) 계산을 위한 클래스 정의
class ContentLoss(nn.Module):
    def __init__(self, target,):
        super(ContentLoss, self).__init__()
        self.target = target.detach()

    def forward(self, input):
        self.loss = F.mse_loss(input, self.target)
        return input

In [11]:
content_layers = ['conv_2', 'conv_4']
style_layers = ['conv_1', 'conv_3', 'conv_5']

# Style Transfer 손실(loss)을 계산하는 함수
def get_losses(cnn, content_img, style_img, noise_image):
    cnn = copy.deepcopy(cnn)
    normalization = Normalization(cnn_normalization_mean, cnn_normalization_std).to(device)
    content_losses = []
    style_losses = []
    
    # 가장 먼저 입력 이미지가 입력 정규화(input normalization)를 수행하도록
    model = nn.Sequential(normalization)

    # 현재 CNN 모델에 포함되어 있는 모든 레이어를 확인하며
    i = 0
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = 'conv_{}'.format(i)
        elif isinstance(layer, nn.ReLU):
            name = 'relu_{}'.format(i)
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'bn_{}'.format(i)
        else:
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

        model.add_module(name, layer)

        # 설정한 content layer까지의 결과를 이용해 content loss를 계산
        if name in content_layers:
            target_feature = model(content_img).detach()
            content_loss = ContentLoss(target_feature)
            model.add_module("content_loss_{}".format(i), content_loss)
            content_losses.append(content_loss)

        # 설정한 style layer까지의 결과를 이용해 style loss를 계산
        if name in style_layers:
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module("style_loss_{}".format(i), style_loss)
            style_losses.append(style_loss)

    # 마지막 loss 이후의 레이어는 사용하지 않도록
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break

    model = model[:(i + 1)]
    return model, content_losses, style_losses

In [12]:
def style_transfer(cnn, content_img, style_img, input_img, iters):
    model, content_losses, style_losses = get_losses(cnn, content_img, style_img, input_img)
    optimizer = optim.LBFGS([input_img.requires_grad_()])

    print("[ Start ]")
    imshow(input_img)

    # 하나의 값만 이용하기 위해 배열 형태로 사용
    run = [0]
    while run[0] <= iters:

        def closure():
            input_img.data.clamp_(0, 1)

            optimizer.zero_grad()
            model(input_img)
            content_score = 0
            style_score = 0

            for cl in content_losses:
                content_score += cl.loss
            for sl in style_losses:
                style_score += sl.loss

            style_score *= 1e5
            loss = content_score + style_score
            loss.backward()

            run[0] += 1
            if run[0] % 100 == 0:
                print(f"[ Step: {run[0]} / Content loss: {content_score.item()} / Style loss: {style_score.item()}]")
                imshow(input_img)
            
            return content_score + style_score
        
        optimizer.step(closure)

    # 결과적으로 이미지의 각 픽셀의 값이 [0, 1] 사이의 값이 되도록 자르기
    input_img.data.clamp_(0, 1)

    return input_img

# Style Transfer

In [13]:
for content_img_folder_name in content_img_folder_list:
    print(content_img_folder_name)
    # 폴더 내의 파일 리스트
    content_img_list = os.listdir(content_img_folder_path + content_img_folder_name)

    #imshow(style_img)
    for content_img_name in content_img_list:
        content_img_size = PIL.Image.open(content_img_folder_path + content_img_folder_name + '/' + content_img_name).size
        content_img = image_loader(content_img_folder_path + content_img_folder_name + '/' + content_img_name, (content_img_size[1], content_img_size[0]))
        #imshow(content_img)

        input_img = torch.empty_like(content_img).uniform_(0, 1).to(device)
        #imshow(input_img)

        style_img = image_loader(style_img_path + style_img_name, (content_img_size[1], content_img_size[0]))

        # style transfer 수행
        output = style_transfer(cnn, content_img=content_img, style_img=style_img, input_img=input_img, iters=1500)

        # 이미지 저장
        save_image(output.cpu().detach()[0], './style_transfer_img/' + style_img_name.split('.')[0] + '/' + content_img_folder_name + '/' + content_img_name.split('.')[0] + '_style1' + '.jpg')
        print('='*60)

carp
[ Start ]
[ Step: 100 / Content loss: 25.150100708007812 / Style loss: 8.642428398132324]
[ Step: 200 / Content loss: 19.893417358398438 / Style loss: 4.367905616760254]
[ Step: 300 / Content loss: 19.23328399658203 / Style loss: 3.1527867317199707]
[ Step: 400 / Content loss: 19.06269073486328 / Style loss: 2.9896862506866455]
[ Step: 500 / Content loss: 18.963529586791992 / Style loss: 2.918612241744995]
[ Step: 600 / Content loss: 18.891550064086914 / Style loss: 2.881519079208374]
[ Step: 700 / Content loss: 18.848896026611328 / Style loss: 2.8492820262908936]
[ Step: 800 / Content loss: 18.81769561767578 / Style loss: 2.833531141281128]
[ Step: 900 / Content loss: 18.79688835144043 / Style loss: 2.8197052478790283]
[ Step: 1000 / Content loss: 18.783626556396484 / Style loss: 2.808410406112671]
[ Step: 1100 / Content loss: 18.768064498901367 / Style loss: 2.8043782711029053]
[ Step: 1200 / Content loss: 18.762245178222656 / Style loss: 2.7927472591400146]
[ Step: 1300 / Conte

[ Step: 800 / Content loss: 30.970260620117188 / Style loss: 9.14099407196045]
[ Step: 900 / Content loss: 13.669414520263672 / Style loss: 4.382986068725586]
[ Step: 1000 / Content loss: 10.083560943603516 / Style loss: 2.292940139770508]
[ Step: 1100 / Content loss: 8.859940528869629 / Style loss: 1.4531396627426147]
[ Step: 1200 / Content loss: 8.468620300292969 / Style loss: 1.3397868871688843]
[ Step: 1300 / Content loss: 8.331801414489746 / Style loss: 1.3103634119033813]
[ Step: 1400 / Content loss: 8.32473373413086 / Style loss: 1.3237786293029785]
[ Step: 1500 / Content loss: 590.4341430664062 / Style loss: 2531889.75]
[ Start ]
[ Step: 100 / Content loss: 21.363969802856445 / Style loss: 7.459278106689453]
[ Step: 200 / Content loss: 12.335588455200195 / Style loss: 4.365055561065674]
[ Step: 300 / Content loss: 10.94765853881836 / Style loss: 2.0313637256622314]
[ Step: 400 / Content loss: 10.275886535644531 / Style loss: 1.4312455654144287]
[ Step: 500 / Content loss: 9.945

[ Start ]
[ Step: 100 / Content loss: 18.45307159423828 / Style loss: 7.4060516357421875]
[ Step: 200 / Content loss: 8.05533218383789 / Style loss: 4.253903865814209]
[ Step: 300 / Content loss: 6.518162250518799 / Style loss: 2.132966995239258]
[ Step: 400 / Content loss: 6.034081935882568 / Style loss: 1.3899801969528198]
[ Step: 500 / Content loss: 5.821408748626709 / Style loss: 1.2924593687057495]
[ Step: 600 / Content loss: 5.739276885986328 / Style loss: 1.2716286182403564]
[ Step: 700 / Content loss: 5.694352149963379 / Style loss: 1.2585618495941162]
[ Step: 800 / Content loss: 5.662574291229248 / Style loss: 1.2589420080184937]
[ Step: 900 / Content loss: 5.644388198852539 / Style loss: 1.2444018125534058]
[ Step: 1000 / Content loss: 5.628210067749023 / Style loss: 1.260546326637268]
[ Step: 1100 / Content loss: 5.6165771484375 / Style loss: 1.237326741218567]
[ Step: 1200 / Content loss: 5.606472969055176 / Style loss: 1.235392689704895]
[ Step: 1300 / Content loss: 5.5983

[ Step: 800 / Content loss: 486.11761474609375 / Style loss: 1696799.125]
[ Step: 900 / Content loss: 36.155914306640625 / Style loss: 13.083374977111816]
[ Step: 1000 / Content loss: 12.178201675415039 / Style loss: 5.237517356872559]
[ Step: 1100 / Content loss: 7.0562052726745605 / Style loss: 2.89768648147583]
[ Step: 1200 / Content loss: 5.766714096069336 / Style loss: 1.391120433807373]
[ Step: 1300 / Content loss: 5.146073341369629 / Style loss: 1.0027488470077515]
[ Step: 1400 / Content loss: 4.937021732330322 / Style loss: 0.9452318549156189]
[ Step: 1500 / Content loss: 4.875566482543945 / Style loss: 0.9282790422439575]
[ Start ]
[ Step: 100 / Content loss: 22.535127639770508 / Style loss: 8.420215606689453]
[ Step: 200 / Content loss: 17.491920471191406 / Style loss: 4.317180156707764]
[ Step: 300 / Content loss: 16.797697067260742 / Style loss: 2.8329296112060547]
[ Step: 400 / Content loss: 16.546894073486328 / Style loss: 2.5734238624572754]
[ Step: 500 / Content loss: 1

[ Start ]
[ Step: 100 / Content loss: 19.06175994873047 / Style loss: 7.046782970428467]
[ Step: 200 / Content loss: 10.596644401550293 / Style loss: 3.9697952270507812]
[ Step: 300 / Content loss: 8.459207534790039 / Style loss: 1.9931743144989014]
[ Step: 400 / Content loss: 7.473092555999756 / Style loss: 1.278250813484192]
[ Step: 500 / Content loss: 6.903217315673828 / Style loss: 1.2540340423583984]
[ Step: 600 / Content loss: 469.2291259765625 / Style loss: 1437910.125]
[ Step: 700 / Content loss: 44.41674041748047 / Style loss: 40.7004508972168]
[ Step: 800 / Content loss: 17.175601959228516 / Style loss: 4.858632564544678]
[ Step: 900 / Content loss: 10.549169540405273 / Style loss: 2.727508783340454]
[ Step: 1000 / Content loss: 7.979855537414551 / Style loss: 1.257068157196045]
[ Step: 1100 / Content loss: 6.941924095153809 / Style loss: 1.2781519889831543]
[ Step: 1200 / Content loss: 6.792867183685303 / Style loss: 1.2915256023406982]
[ Step: 1300 / Content loss: 569.81835

...

[ Step: 1100 / Content loss: 9.988794326782227 / Style loss: 1.4137563705444336]
[ Step: 1200 / Content loss: 9.932722091674805 / Style loss: 1.4189972877502441]
[ Step: 1300 / Content loss: 569.114990234375 / Style loss: 2277391.5]
[ Step: 1400 / Content loss: 22.503137588500977 / Style loss: 6.921723365783691]
[ Step: 1500 / Content loss: 13.202438354492188 / Style loss: 4.406749725341797]
[ Start ]
[ Step: 100 / Content loss: 16.96324920654297 / Style loss: 7.694461345672607]
[ Step: 200 / Content loss: 7.262171745300293 / Style loss: 4.364773273468018]
[ Step: 300 / Content loss: 5.717087745666504 / Style loss: 2.230771064758301]
[ Step: 400 / Content loss: 5.409655570983887 / Style loss: 1.4935883283615112]
[ Step: 500 / Content loss: 5.279015064239502 / Style loss: 1.286258339881897]
[ Step: 600 / Content loss: 5.215519905090332 / Style loss: 1.2303451299667358]
[ Step: 700 / Content loss: 5.148603439331055 / Style loss: 1.2026090621948242]
[ Step: 800 / Content loss: 472.4166259

[ Step: 300 / Content loss: 8.724382400512695 / Style loss: 2.002368688583374]
[ Step: 400 / Content loss: 7.591686725616455 / Style loss: 1.3297805786132812]
[ Step: 500 / Content loss: 7.069672584533691 / Style loss: 1.2743723392486572]
[ Step: 600 / Content loss: 6.885829448699951 / Style loss: 1.2662608623504639]
[ Step: 700 / Content loss: 6.805337429046631 / Style loss: 1.2778253555297852]
[ Step: 800 / Content loss: 6.767022132873535 / Style loss: 1.2855738401412964]
[ Step: 900 / Content loss: 7.172457695007324 / Style loss: 1.4121503829956055]
[ Step: 1000 / Content loss: 6.672342777252197 / Style loss: 1.258080005645752]
[ Step: 1100 / Content loss: 6.7251176834106445 / Style loss: 1.267334222793579]
[ Step: 1200 / Content loss: 6.654251575469971 / Style loss: 1.2588188648223877]
[ Step: 1300 / Content loss: 551.2230224609375 / Style loss: 2132069.25]
[ Step: 1400 / Content loss: 25.919105529785156 / Style loss: 8.167015075683594]
[ Step: 1500 / Content loss: 11.8573226928710

[ Step: 1100 / Content loss: 10.92302417755127 / Style loss: 0.9832156896591187]
[ Step: 1200 / Content loss: 10.9511079788208 / Style loss: 1.1088296175003052]
[ Step: 1300 / Content loss: 603.8978271484375 / Style loss: 2435210.75]
[ Step: 1400 / Content loss: 33.32725524902344 / Style loss: 17.337419509887695]
[ Step: 1500 / Content loss: 17.414297103881836 / Style loss: 5.2842116355896]
[ Start ]
[ Step: 100 / Content loss: 34.47441101074219 / Style loss: 9.090644836425781]
[ Step: 200 / Content loss: 25.03565216064453 / Style loss: 5.7271037101745605]
[ Step: 300 / Content loss: 24.02944564819336 / Style loss: 3.5307445526123047]
[ Step: 400 / Content loss: 23.840850830078125 / Style loss: 2.8357443809509277]
[ Step: 500 / Content loss: 23.6708927154541 / Style loss: 2.639591693878174]
[ Step: 600 / Content loss: 23.54666519165039 / Style loss: 2.5985264778137207]
[ Step: 700 / Content loss: 23.528379440307617 / Style loss: 3.4543657302856445]
[ Step: 800 / Content loss: 23.368997

[ Step: 300 / Content loss: 18.07657814025879 / Style loss: 2.4272899627685547]
[ Step: 400 / Content loss: 17.5764102935791 / Style loss: 2.2601401805877686]
[ Step: 500 / Content loss: 17.382183074951172 / Style loss: 2.229945182800293]
[ Step: 600 / Content loss: 17.255455017089844 / Style loss: 2.217092514038086]
[ Step: 700 / Content loss: 17.124330520629883 / Style loss: 2.34355092048645]
[ Step: 800 / Content loss: 17.056886672973633 / Style loss: 2.2534663677215576]
[ Step: 900 / Content loss: 16.982452392578125 / Style loss: 2.2538466453552246]
[ Step: 1000 / Content loss: 17.02871322631836 / Style loss: 2.3026840686798096]
[ Step: 1100 / Content loss: 16.874046325683594 / Style loss: 2.262877941131592]
[ Step: 1200 / Content loss: 16.843069076538086 / Style loss: 2.237274646759033]
[ Step: 1300 / Content loss: 546.654052734375 / Style loss: 1846197.875]
[ Step: 1400 / Content loss: 33.293575286865234 / Style loss: 12.386467933654785]
[ Step: 1500 / Content loss: 22.2371788024

[ Step: 1000 / Content loss: 21.849382400512695 / Style loss: 2.1873104572296143]
[ Step: 1100 / Content loss: 21.818689346313477 / Style loss: 2.1829774379730225]
[ Step: 1200 / Content loss: 21.79269790649414 / Style loss: 2.175954580307007]
[ Step: 1300 / Content loss: 21.770416259765625 / Style loss: 2.173145294189453]
[ Step: 1400 / Content loss: 21.748088836669922 / Style loss: 2.1896603107452393]
[ Step: 1500 / Content loss: 21.730670928955078 / Style loss: 2.1697583198547363]
[ Start ]
[ Step: 100 / Content loss: 24.035049438476562 / Style loss: 8.001774787902832]
[ Step: 200 / Content loss: 14.417115211486816 / Style loss: 4.304551601409912]
[ Step: 300 / Content loss: 12.949264526367188 / Style loss: 2.0808265209198]
[ Step: 400 / Content loss: 12.346171379089355 / Style loss: 1.4370664358139038]
[ Step: 500 / Content loss: 11.965538024902344 / Style loss: 1.3147019147872925]
[ Step: 600 / Content loss: 11.81460189819336 / Style loss: 1.2983478307724]
[ Step: 700 / Content lo